In [1]:
import mst_mach_tools as mmt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')
import MDSplus as mds
import pandas as pd
import numpy as np
import numpy.linalg as nl

In [2]:
shot_file = "/home/reksoatr/canonical_vorticity_probe/tetra/tetrahedral prototype commissioning - 2022-09 - shot settings.xlsx"
compressed_shot_settings = pd.read_excel(shot_file, skiprows=2, header=0)
shot_settings = []
for i, shot in enumerate(compressed_shot_settings['Shots']): 
    if str(shot) == 'nan':
        continue
    if '-' not in shot:
        shot_settings.append(compressed_shot_settings.iloc[i].to_dict())
    if '-' in shot:
        shot_start, shot_end = shot.split('-')
        date = shot_start[:-3]
        start = shot_start[-3:]
        for j in range(int(start), int(shot_end) + 1):
            shot_to_save = date + str(j).zfill(3)
            settings = compressed_shot_settings.iloc[i].to_dict()
            settings['Shots'] = shot_to_save
            shot_settings.append(settings)
shot_settings = pd.DataFrame(shot_settings)

In [3]:
shot_settings

,Shots,Location (cm),Orientation,return,RJ1 (Ohm),RJ2 (Ohm),RJ3 (Ohm),RJ4 (Ohm),RJ return,Bias Bank (V),...,Bias 3 (V),Bias 4 (V),Gain 1,Gain 2,Gain 3,Gain 4,Gain B1 p,Gain B2 r,Gain B3 t,Notes
0,1220926011,3.0,0.0,electrode,10.0,100.0,100.0,50.0,20.0,80.0,...,NaN,NaN,1.0,1.0,1.0,1.0,50.0,50.0,50.0,Bcoil amps off
1,1220926012,3.0,0.0,electrode,10.0,100.0,100.0,50.0,20.0,80.0,...,NaN,NaN,1.0,1.0,1.0,1.0,50.0,50.0,50.0,Bcoil amps off
2,1220926013,3.0,0.0,electrode,10.0,100.0,100.0,50.0,20.0,80.0,...,NaN,NaN,1.0,1.0,1.0,1.0,50.0,50.0,50.0,Bcoil amps off
3,1220926014,3.0,0.0,electrode,10.0,100.0,100.0,50.0,20.0,80.0,...,NaN,NaN,1.0,1.0,1.0,1.0,50.0,50.0,50.0,shot 019 No plasma
4,1220926015,3.0,0.0,electrode,10.0,100.0,100.0,50.0,20.0,80.0,...,NaN,NaN,1.0,1.0,1.0,1.0,50.0,50.0,50.0,shot 019 No plasma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,1220929058,5.0,270.0,vessel,20.0,20.0,20.0,NaN,5.0,NaN,...,166.0,190.0,1.0,1.0,1.0,1.0,50.0,50.0,50.0,NaN
266,1220929059,5.0,270.0,vessel,20.0,20.0,20.0,NaN,5.0,NaN,...,166.0,190.0,1.0,1.0,1.0,1.0,50.0,50.0,50.0,NaN
267,1220929060,5.0,270.0,vessel,20.0,20.0,20.0,NaN,5.0,NaN,...,166.0,190.0,1.0,1.0,1.0,1.0,50.0,50.0,50.0,NaN
268,1220929061,5.0,270.0,vessel,20.0,20.0,20.0,NaN,5.0,NaN,...,166.0,190.0,1.0,1.0,1.0,1.0,50.0,50.0,50.0,NaN


In [4]:
shot_settings.keys()

Index(['Shots', 'Location (cm)', 'Orientation', 'return', 'RJ1 (Ohm)',
       'RJ2  (Ohm)', 'RJ3  (Ohm)', 'RJ4  (Ohm)', 'RJ return', 'Bias Bank (V)',
       'Bias 1 (V)', 'Bias 2 (V)', 'Bias 3 (V)', 'Bias 4 (V)', 'Gain 1',
       'Gain 2', 'Gain 3', 'Gain 4', 'Gain B1 p', 'Gain B2 r', 'Gain B3 t',
       'Notes'],
      dtype='object')

In [5]:
shot_settings[shot_settings['Shots'] == '1220927038']['RJ2  (Ohm)'].values[0]

20.0

In [20]:
"START 'mach_results' METHOD PROCESS"

import mst_mach_tools as mmt
import matplotlib.pyplot as plt
import numpy as np

shot='1220927056'
nsmooth=None
res='mach'
delphi=-60.0
tmin=-0.001
tmax=0.07
comp='toni'
vtype='tetrahedron'
imin=5E-4
k=1.1
second_shot='1220927038'
tetra_missing_tip=4
mds_start='probe5_'
merge_method='average'

RJkeys=['RJ1 (Ohm)','RJ2  (Ohm)','RJ3  (Ohm)','RJ4  (Ohm)','RJ return']

rj_set=[]
rj_set2 =[]

for RJ in RJkeys:
    rj_set.append(shot_settings[shot_settings['Shots'] == shot][RJ].values[0])
    rj_set2.append(shot_settings[shot_settings['Shots'] == second_shot][RJ].values[0])

rj_set, rj_set2

([20.0, 20.0, 20.0, nan, 5.0], [20.0, 20.0, 20.0, nan, 5.0])

In [8]:
isat = {}
time = {}
isat2 = {}
time2 = {}
if vtype.startswith('octa'):
    tip_abc_name = ['a', 'b', 'c', 'd', 'e', 'f', 'ret']
    tip_num_name = ['1', '4', '5', '2', '3', '6', '7']
if vtype.startswith('tetra'):
    tip_abc_name = ['a', 'b', 'c', 'd', 'ret']
    #second_tip_abc_name = ['b', 'a', 'd', 'c', 'ret']
    tip_num_name = ['4', '1', '3', '2', '5']
    tip_num_name2 = ['3', '2', '4', '1', '5']

In [9]:
rj = dict(zip(tip_abc_name,
     rj_set))
rj2 = dict(zip(tip_abc_name,
      rj_set2))
tip_mds_name = [mds_start + str(n).zfill(2) for n in \
    tip_num_name]   
tip_mds_name2 = [mds_start + str(n).zfill(2) for n in \
    tip_num_name2]
mst_name = ['theta', 'phi', 'r']  # ['x', 'y', 'z'] in Cartesian
rj, rj2, tip_mds_name, tip_mds_name2

({'a': 20.0, 'b': 20.0, 'c': 20.0, 'd': nan, 'ret': 5.0},
 {'a': 20.0, 'b': 20.0, 'c': 20.0, 'd': nan, 'ret': 5.0},
 ['probe5_04', 'probe5_01', 'probe5_03', 'probe5_02', 'probe5_05'],
 ['probe5_03', 'probe5_02', 'probe5_04', 'probe5_01', 'probe5_05'])

In [21]:
for i in range(len(tip_abc_name)):
    abc = tip_abc_name[i]
    time[abc], isat[abc] = mmt.get_data(name=tip_mds_name[i], shot=int(shot),
        comp=comp, tmin=tmin, tmax=tmax)
    #
    # if second_shot assume it is with probe rotated by 180 degrees
    if second_shot:
        time2[abc], isat2[abc] = mmt.get_data(name=tip_mds_name2[i], shot=int(second_shot),
                                          comp=comp, tmin=tmin, tmax=tmax)

        if nsmooth is not None:
            isat2[abc] = mmt.smooth(isat2[abc], nsmooth)
        isat2[abc] /= rj2[abc]
        isat2[abc] = np.maximum(isat2[abc], imin)

    if nsmooth is not None:
        isat[abc] = mmt.smooth(isat[abc], nsmooth)
    isat[abc] /= rj[abc]
    isat[abc] = np.maximum(isat[abc], imin)

if second_shot:
    time, isat = mmt.merge_shots(time, time2, isat, isat2,
                             tip_abc_name[tip_num_name.index(str(tetra_missing_tip))],
                             tip_abc_name[tip_num_name2.index(str(tetra_missing_tip))],
                             merge_method=merge_method)

    if merge_method == 'second':
        tip_num_name = tip_num_name2

In [23]:
isat2

{'a': array([0.0005, 0.0005, 0.0005, ..., 0.0005, 0.0005, 0.0005], dtype=float32),
 'b': array([0.003663  , 0.0041514 , 0.0034188 , ..., 0.00586081, 0.00573871,
        0.00610501], dtype=float32),
 'c': array([0.00695971, 0.00610501, 0.00757021, ..., 0.00695971, 0.00708181,
        0.00695971], dtype=float32),
 'd': array([nan, nan, nan, ..., nan, nan, nan], dtype=float32),
 'ret': array([0.0043956 , 0.00634921, 0.0039072 , ..., 0.00586081, 0.00634921,
        0.0043956 ], dtype=float32)}

In [99]:
for i in range(len(tip_abc_name)):
    abc = tip_abc_name[i]
    if abc == 'a':
        isat['sum'] = np.zeros(np.shape(isat['a']))
    if abc != 'ret':
        isat['sum'] += isat[abc]

In [ ]:
time_arrays_equal = True
for name in tip_abc_name[1:]:
    if np.array_equal(time['a'], time[name]):
        continue
    time_arrays_equal = False
if (time_arrays_equal):
    time = time['a']
else:
    print('Time arrays not all equal')
    return None
if res.lower().startswith('isat'):
    isat['t'] = time
    isat['name'] = dict(zip(tip_num_name, tip_abc_name))
    isat['rj'] = rj
    isat['units'] = 'A'
    return isat
if vtype.startswith('octa'):
    ratio = np.array((isat['a']/isat['d'], isat['b']/isat['e'],
        isat['c']/isat['f']))
    rat_name = ['a/d', 'b/e', 'c/f']
elif vtype.startswith('tetra'):        
    ratio = np.array((isat['a']*isat['b']/isat['c']/isat['d'],
        isat['b']*isat['c']/isat['d']/isat['a'],
        isat['c']*isat['a']/isat['b']/isat['d']))
    rat_name = ['a*b/c/d', 'b*c/d/a', 'c*a/b/d']
if res.startswith('rat'):
    return {'rat':ratio, 't':time, 'name':rat_name, 'units':None}
log = - 1.0/k*np.log(ratio)
if res.startswith('log'):
    return {'log':log, 't':time, 'name':rat_name, 'units':None}
mat = angle_matrix(vtype=vtype, rz=delphi)
if res.startswith('mat'):
    return {'mat':mat, 'mst_name':mst_name, 'ratio_name':rat_name}
inv = nl.inv(mat)
if res.startswith('inv'):
    return {'inv':inv, 'mst_name':mst_name, 'ratio_name':rat_name}
vel = np.dot(inv, log)
if res.startswith('mach'):
    return {'t':time, 'r':vel[mst_name.index('r')],
        'theta':vel[mst_name.index('theta')],
        'phi':vel[mst_name.index('phi')], 'units':'M'}